# Finetuning T5

The purpose of this notebook is to document the process of finetuning Google's T5 model for translating from Literary Tibetan to English. This notebook relies on a dataset in the form of a pickled pandas dataframe which consists of a single column, 'translation'. Entries in that column should be a python dictionary of the structure: {'bo':'Tibetan text', 'en': 'English text'}.

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('pandas', data_files='/home/j/Documents/Projects/MLotsawa/notebooks/t5/sample-data.p')

/home/j/Documents/Projects/MLotsawa/.venv/lib/python3.12/site-packages/datasets/packaged_modules/pandas/pandas.py:27: FutureWarning: The Pandas builder is deprecated and will be removed in the next major version of datasets.
  warnings.warn(


In [3]:
dataset['train']

Dataset({
    features: ['translation'],
    num_rows: 2
})

In [4]:
dataset = dataset['train'].train_test_split(test_size=.5)

In [5]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/j/Documents/Projects/MLotsawa/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
source_lang = 'bo'
target_lang = 'en'
prefix = "translate Tibetan to English: "

def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples['translation']]
    targets = [example[target_lang] for example in examples['translation']]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

    return model_inputs


In [7]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [9]:
import evaluate

metric = evaluate.load("sacrebleu")

In [10]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/home/j/Documents/Projects/MLotsawa/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/2 [00:00<?, ?it/s]

{'train_runtime': 1.5191, 'train_samples_per_second': 1.317, 'train_steps_per_second': 1.317, 'train_loss': 7.118500709533691, 'epoch': 2.0}


TrainOutput(global_step=2, training_loss=7.118500709533691, metrics={'train_runtime': 1.5191, 'train_samples_per_second': 1.317, 'train_steps_per_second': 1.317, 'total_flos': 4229431296.0, 'train_loss': 7.118500709533691, 'epoch': 2.0})